In [ ]:
import os
import cv2
import tqdm
import numpy as np
from skimage.io import imread, imshow

In [ ]:
def load_data(datapath='lane_marking_examples', load_from=0, load_count=-1, resize_coef=4):
    filenames = []
    for top, dirs, files in os.walk(datapath):
        filenames.extend([os.path.join(top, _file) for _file in files])
    filenames.sort()

    x_paths = [x for x in filenames if not x.endswith('bin.png')]
    y_paths = [x for x in filenames if x.endswith('bin.png')]

    if load_count != -1:
        x_paths = x_paths[load_from:load_from + load_count]
        y_paths = y_paths[load_from:load_from + load_count]
    
    pic_w, pic_h, pic_ch = cv2.imread(x_paths[0]).shape
    
    if resize_coef != 1:
        pic_w, pic_h = int(pic_w / resize_coef), int(pic_h / resize_coef)
    
    xdata = np.zeros((len(x_paths), pic_w, pic_h, pic_ch), dtype=np.uint8)
    ydata = np.zeros((len(x_paths), pic_w, pic_h, pic_ch), dtype=np.uint8)

    xdata_size = round(xdata.nbytes / 1024 / 1024 / 1024, 2)
    ydata_size = round(ydata.nbytes / 1024 / 1024 / 1024, 2)
    
    print(f'XData size: {xdata_size} GB,\nYData size: {ydata_size} GB\nTotal: {xdata_size} GB')
    if resize_coef == 1:
        for ind, x in enumerate(tqdm.tqdm_notebook(x_paths)):
            xdata[ind] = cv2.imread(x)

        for ind, y in enumerate(tqdm.tqdm_notebook(y_paths)):
            ydata[ind] = cv2.imread(y)
    else:
        for ind, x in enumerate(tqdm.tqdm_notebook(x_paths)):
            xdata[ind] = cv2.resize(cv2.imread(x), (pic_h, pic_w), interpolation=cv2.INTER_NEAREST)

        for ind, y in enumerate(tqdm.tqdm_notebook(y_paths)):
            ydata[ind] = cv2.resize(cv2.imread(y), (pic_h, pic_w), interpolation=cv2.INTER_NEAREST)
        
    return xdata, ydata

In [ ]:
xdata, ydata = load_data()

In [ ]:
imshow(np.concatenate((xdata[0], ydata[0]), axis=1));

In [ ]:
unqs = []
for lbl in tqdm.tqdm_notebook(ydata):
    _u = np.unique(lbl.reshape(-1, 3), axis=0)
    for e in _u:
        if tuple(e) not in unqs:
            unqs.append(tuple(e))
            print(len(unqs), unqs)
    # _u = set([tuple(x) for x in np.unique(lbl.reshape(-1, 3), axis=0).tolist()])
    # unqs.update(_u)
unqs

In [ ]:
# Label colors =======
# labels = {
#     (153, 102, 153): ['zebra', 214], # Crossing
#     (180, 173, 43): [],  # Road side Lane
#     (0, 0, 192): [], # Stop before crossing
#     (142, 35, 8): [], # Go ahead arrow
#     (160, 168, 234): [] # Go ahead and left turn arrow}
labels = [(0, 0, 0), (8, 35, 142), (43, 173, 180), 
          (153, 102, 153), (255, 255, 255), (192, 0, 0), 
          (8, 32, 128), (12, 51, 204), (70, 25, 100), 
          (14, 57, 230), (75, 47, 190), (234, 168, 160)]

# color_codes = dict([(k, [0 for i in range(12)]) for k in labels])
# for k in color_codes.keys():
#     color_codes[k][list(color_codes.keys()).index(k)] = 1

labels = {
 (0, 0, 0): [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 (8, 35, 142): [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 (43, 173, 180): [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 (153, 102, 153): [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 (255, 255, 255): [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 (192, 0, 0): [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 (8, 32, 128): [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 (12, 51, 204): [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 (70, 25, 100): [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 (14, 57, 230): [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 (75, 47, 190): [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 (234, 168, 160): [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]}
labels = dict([(k, np.array(v)) for k, v in labels.items()])

img_h, img_w = xdata[0].shape[:2]

labels_converted = np.zeros((len(ydata[:1000]), img_h, img_w, len(labels)))
for i in tqdm.tqdm_notebook(range(len(labels_converted))):
    for rgb, categorical_lbl in labels.items():
        # labels_converted[i, :, :, labels.index(m)] = `
        # np.apply_along_axis(lambda x: np.all(x), axis=2, arr=ydata[i] == m)
        labels_converted[i, (ydata[i] == rgb).all(2)] = categorical_lbl

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]='PCI_BUS_ID'   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]='1'

import json
from keras import models
from keras.layers.core import Activation, Reshape, Permute
from keras.layers.convolutional import Conv2D, MaxPooling2D, UpSampling2D
from keras.layers.normalization import BatchNormalization


class SegnetBuilder:
    @staticmethod
    def build(model_name, img_h, img_w, img_layers, n_labels, kernel=3,
              save_path='models/{}.json') -> models.Sequential:
        encoding_layers = [
            Conv2D(64, kernel, padding='same', input_shape=(img_w, img_h, img_layers)),
            BatchNormalization(),
            Activation('relu'),
            Conv2D(64, kernel, padding='same'),
            BatchNormalization(),
            Activation('relu'),
            MaxPooling2D(),

            Conv2D(128, (kernel, kernel), padding='same'),
            BatchNormalization(),
            Activation('relu'),
            Conv2D(128, (kernel, kernel), padding='same'),
            BatchNormalization(),
            Activation('relu'),
            MaxPooling2D(),

            Conv2D(256, (kernel, kernel), padding='same'),
            BatchNormalization(),
            Activation('relu'),
            Conv2D(256, (kernel, kernel), padding='same'),
            BatchNormalization(),
            Activation('relu'),
            Conv2D(256, (kernel, kernel), padding='same'),
            BatchNormalization(),
            Activation('relu'),
            MaxPooling2D(),

            Conv2D(512, (kernel, kernel), padding='same'),
            BatchNormalization(),
            Activation('relu'),
            Conv2D(512, (kernel, kernel), padding='same'),
            BatchNormalization(),
            Activation('relu'),
            Conv2D(512, (kernel, kernel), padding='same'),
            BatchNormalization(),
            Activation('relu'),
            MaxPooling2D(),

            Conv2D(512, (kernel, kernel), padding='same'),
            BatchNormalization(),
            Activation('relu'),
            Conv2D(512, (kernel, kernel), padding='same'),
            BatchNormalization(),
            Activation('relu'),
            Conv2D(512, (kernel, kernel), padding='same'),
            BatchNormalization(),
            Activation('relu'),
            MaxPooling2D(),
            ]

        decoding_layers = [
            UpSampling2D(),
            Conv2D(512, (kernel, kernel), padding='same'),
            BatchNormalization(),
            Activation('relu'),
            Conv2D(512, (kernel, kernel), padding='same'),
            BatchNormalization(),
            Activation('relu'),
            Conv2D(512, (kernel, kernel), padding='same'),
            BatchNormalization(),
            Activation('relu'),

            UpSampling2D(),
            Conv2D(512, (kernel, kernel), padding='same'),
            BatchNormalization(),
            Activation('relu'),
            Conv2D(512, (kernel, kernel), padding='same'),
            BatchNormalization(),
            Activation('relu'),
            Conv2D(256, (kernel, kernel), padding='same'),
            BatchNormalization(),
            Activation('relu'),

            UpSampling2D(),
            Conv2D(256, (kernel, kernel), padding='same'),
            BatchNormalization(),
            Activation('relu'),
            Conv2D(256, (kernel, kernel), padding='same'),
            BatchNormalization(),
            Activation('relu'),
            Conv2D(128, (kernel, kernel), padding='same'),
            BatchNormalization(),
            Activation('relu'),

            UpSampling2D(),
            Conv2D(128, (kernel, kernel), padding='same'),
            BatchNormalization(),
            Activation('relu'),
            Conv2D(64, (kernel, kernel), padding='same'),
            BatchNormalization(),
            Activation('relu'),

            UpSampling2D(),
            Conv2D(64, (kernel, kernel), padding='same'),
            BatchNormalization(),
            Activation('relu'),
            Conv2D(n_labels, (1, 1), padding='valid'),
            BatchNormalization(),
            ]

        autoencoder = models.Sequential()
        autoencoder.encoding_layers = encoding_layers

        for l in autoencoder.encoding_layers:
            autoencoder.add(l)
            # print(l.input_shape, l.output_shape, l)

        autoencoder.decoding_layers = decoding_layers
        for l in autoencoder.decoding_layers:
            autoencoder.add(l)

        autoencoder.add(Reshape((n_labels, img_h * img_w)))
        autoencoder.add(Permute((2, 1)))
        autoencoder.add(Activation('softmax'))

        with open(save_path.format(model_name), 'w') as outfile:
            outfile.write(json.dumps(json.loads(autoencoder.to_json()), indent=2))

        return autoencoder

In [ ]:
segnet = SegnetBuilder.build('Segnet-1000', img_h, img_w, 3, len(labels))

segnet.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_acc', min_delta=0.0001,
                           patience=10, verbose=1, mode='auto')
checkpoint = ModelCheckpoint(f'models/{self.model_name}.hdf5',
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='auto')

callbacks = [early_stop, checkpoint]

history = autoencoder.fit(xdata[:1000], labels_converted, batch_size=16, epochs=20,
                          verbose=1, validation_split=0.2, callbacks=callbacks)